# Tokens de pensamento (palavras de linguagem natural)

In [ ]:
print("here we go!")

here we go!


Este capítulo cobre

* Analisando seu texto em palavras en -gramas (tokens)
* Tokenização de pontuação, emoticons e até caracteres chineses
* Consolidando seu vocabulário com lematização, lematização e dobramento de maiúsculas e minúsculas
* Construindo uma representação numérica estruturada de texto em linguagem natural
* Pontuação do texto quanto ao sentimento e intenção pró-social
* Usando análise de frequência de caracteres para otimizar seu vocabulário de tokens
* Lidando com sequências de palavras e tokens de comprimento variável

## **O tokenizador mais simples**



In [ ]:
# split
text = ("Trust me, though, the words were on their way, and when "
         "they arrived, Liesel would hold them in her hands like "
         "the clouds, and she would wring them out, like the rain.")
tokens = text.split()
tokens[:8]


['Trust', 'me,', 'though,', 'the', 'words', 'were', 'on', 'their']

Como você pode ver, este método Python integrado faz um bom trabalho ao tokenizar esta frase. Seu único “erro” é incluir vírgulas nos tokens. Isso impediria que seu detector de palavras-chave detectasse alguns tokens importantes: ['me', 'though', 'way', 'arrived', 'clouds', 'out', "rain"].

## **Tokenização baseada em regras**

Expressões Regulares

In [ ]:
import re
pattern = r'\w+(?:\'\w+)?|[^\w\s]'

texts = [text]
texts.append("There's no such thing as survival of the fittest. "
              "Survival of the most adequate, maybe.")

tokens = list(re.findall(pattern, texts[-1]))
tokens[:8]

["There's", 'no', 'such', 'thing', 'as', 'survival', 'of', 'the']

In [ ]:
tokens[8:16]

['fittest', '.', 'Survival', 'of', 'the', 'most', 'adequate', ',']

In [ ]:
tokens[16:]

['maybe', '.']

Muito melhor. Agora, o tokenizer separa a pontuação do final de uma palavra, mas não separa palavras que contenham pontuação interna, como o apóstrofo dentro do token "There's"


**Dica profissional: você pode acomodar contrações duplas com a expressão regular**

`r'\w+(?:\'\w+){0,2}|[^\w\s]'`

In [ ]:
import numpy as np
vocab = sorted(set(tokens))
' '.join(vocab[:12]) # Classificado lexograficamente (lexicalmente), de forma que a pontuação venha antes das letras e as letras maiúsculas antes das letras minúsculas.

", . Survival There's adequate as fittest maybe most no of such"

In [ ]:
num_tokens = len(tokens)
num_tokens

18

In [ ]:
vocab_size = len(vocab)
vocab_size

15

DICA: Certifique-se de dar uma olhada em seu vocabulário sempre que parecer que seu pipeline não está funcionando bem para um texto específico. Pode ser necessário revisar seu tokenizer para ter certeza de que ele pode "ver" todos os tokens necessários para um bom desempenho em sua tarefa de PNL.

## **SpaCy**


Talvez você não queira que seu tokenizador de expressão regular mantenha as contrações juntas. Talvez você queira reconhecer a palavra "isn't" como duas palavras separadas, "is" e "n't". Dessa forma, você poderia consolidar os sinônimos "n't" e "not" em um único token. Dessa forma, seu pipeline de NLP entenderia que "the ice cream isn't bad" significa a mesma coisa que "the ice cream is not bad".

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

doc = nlp(texts[-1])

type(doc)

spacy.tokens.doc.Doc

In [ ]:
tokens = [tok.text for tok in doc]
tokens[:9]

['There', "'s", 'no', 'such', 'thing', 'as', 'survival', 'of', 'the']

In [ ]:
tokens[9:17]

['fittest', '.', 'Survival', 'of', 'the', 'most', 'adequate', ',']

Você pode ver que o spaCy faz muito mais do que simplesmente separar o texto em tokens. Ele identifica os limites das frases para segmentar automaticamente seu texto em frases. E marca tokens com vários atributos, como classe gramatical (PoS) e até mesmo sua função na sintaxe de uma frase

## **Tokenizadores de Palavras**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,2), analyzer='char')
vectorizer.fit(texts)

CountVectorizer(analyzer='char', ngram_range=(1, 2))

CountVectorizer = Converta uma coleção de documentos de texto em uma matriz de contagens de tokens.

* `ngram_range` tuple (min_n, max_n), default=(1, 1)
Os limites inferior e superior do intervalo de valores n para diferentes palavra n-gramas ou char n-gramas a serem extraídos.

* `analyzer` {‘word’, ‘char’, ‘char_wb’} or callable, default=’word’
Se o recurso deve ser feito de palavra n-grama ou caractere n-gramas.


In [ ]:
bpevocab_list = [
    sorted((i, s) for s, i in vectorizer.vocabulary_.items())]
bpevocab_dict = dict(bpevocab_list[0])
list(bpevocab_dict.values())[:7]


[' ', ' a', ' c', ' f', ' h', ' i', ' l']

Nós configuramos o CountVectorizerpara dividir o texto em todos os caracteres possíveis de 1 grama e 2 gramas encontrados nos textos. E CountVectorizerorganiza o vocabulário em ordem lexical, então n-gramas que começam com um caractere de espaço ( ' ') vem primeiro. Uma vez que o vetorizador saiba quais tokens ele precisa para poder contar, ele pode transformar strings de texto em vetores, com uma dimensão para cada token em seu vocabulário de n-gramas de caracteres.

In [ ]:
vectors = vectorizer.transform(texts)
df = pd.DataFrame(
    vectors.todense(),
    columns=vectorizer.vocabulary_)

df.index = [t[:8] + ' ...' for t in texts]
df = df.T
df[' total'] = df.T.sum()
df

,Trust me ...,There's ...,total
t,31,14,45
r,3,2,5
u,1,0,1
s,0,1,1
,3,0,3
...,...,...,...
at,1,0,1
ma,2,1,3
yb,1,0,1
be,1,0,1


In [ ]:
df.sort_values(' total').tail()

,Trust me ...,There's ...,total
en,10,3,13
an,14,5,19
uc,11,9,20
e,18,8,26
t,31,14,45


In [ ]:
df[' n'] = [len(tok) for tok in vectorizer.vocabulary_]
df[df[' n'] > 1].sort_values(' total').tail()

,Trust me ...,There's ...,total,n
ur,8,4,12,2
en,10,3,13,2
an,14,5,19,2
uc,11,9,20,2
e,18,8,26,2


## **Vetores de Tokens**

In [ ]:
hi_text = ' Hiking home now'
hi_text.startswith(' Hi')


True

In [ ]:
pattern = r' \w +(?: \' \w +)?|[^ \w \s ]'
'Hi' in re.findall(pattern, hi_text)


False

### One hot vectors


In [ ]:
import pandas as pd
onehot_vectors = np.zeros(
    (len(tokens), vocab_size), int)

for i, tok in enumerate(tokens):
  if tok not in vocab:
    continue
  onehot_vectors[i, vocab.index(tok)] = 1


df_onehot = pd.DataFrame(onehot_vectors, columns=vocab)
df_onehot.shape


(19, 15)

In [ ]:
df_onehot.iloc[:, :8].replace(0, '')

,",",.,Survival,There's,adequate,as,fittest,maybe
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,,,,,,,,
5,,,,,,1,,
6,,,,,,,,
7,,,,,,,,
8,,,,,,,,
9,,,,,,,1,


### Spacy



In [ ]:
import spacy  #
nlp = spacy.load('en_core_web_sm')  #
nlp


In [ ]:

doc = nlp(texts[-1])
type(doc)


spacy.tokens.doc.Doc

In [ ]:
tokens = [tok.text for tok in doc]  #
tokens[:9]

['There', "'s", 'no', 'such', 'thing', 'as', 'survival', 'of', 'the']

In [ ]:
tokens[9:17]

['fittest', '.', 'Survival', 'of', 'the', 'most', 'adequate', ',']

### Treebank

O tokenizer Treebank usa expressões regulares para tokenizar texto como no Penn Treebank.

Este tokenizer executa as seguintes etapas:

dividir contrações padrão, por exemplo don't-> do n'te they'll-> they 'll

trate a maioria dos caracteres de pontuação como tokens separados

separe vírgulas e aspas simples, quando seguidas de espaço em branco

períodos separados que aparecem no final da linha



In [ ]:
import nltk

In [ ]:
from nltk.tokenize import TreebankWordTokenizer

In [ ]:
s = '''Good muffins cost $3.88\nin New York.  Please buy me\ntwo of them.\nThanks.'''
s

'Good muffins cost $3.88\nin New York.  Please buy me\ntwo of them.\nThanks.'

In [ ]:
TreebankWordTokenizer().tokenize(s)

['Good',
 'muffins',
 'cost',
 '$',
 '3.88',
 'in',
 'New',
 'York.',
 'Please',
 'buy',
 'me',
 'two',
 'of',
 'them.',
 'Thanks',
 '.']

In [ ]:
s = "They'll save and invest more."
TreebankWordTokenizer().tokenize(s)

['They', "'ll", 'save', 'and', 'invest', 'more', '.']

In [ ]:
s = "hi, my name can't hello,"
TreebankWordTokenizer().tokenize(s)

['hi', ',', 'my', 'name', 'ca', "n't", 'hello', ',']

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
text = "I would like a coffee with milk."  #
doc = nlp(text)
for token in doc:
     print(f"{token.text:<11}{token.pos_:<10}{token.dep:<10}")


I          PRON      429       
would      AUX       405       
like       VERB      8206900633647566924
a          DET       415       
coffee     NOUN      416       
with       ADP       443       
milk       NOUN      439       
.          PUNCT     445       


## Tokens Desafiadores


A biblioteca NLTK inclui um tokenizer baseado em regras para lidar com textos curtos, informais e repletos de emojis de redes sociais: casual_tokenize

Ele lida com emojis, emoticons e nomes de usuário. O reduce_lenopção exclui repetições de caracteres menos significativas. O reduce_lenalgoritmo retém três repetições, para aproximar a intenção e o sentimento do texto original.

In [ ]:
from nltk.tokenize.casual import casual_tokenize
text = "@rickrau mind BLOOOOOOOOWWWWWN by latest lex :*) !!!!!!!!"
texts = [text]
casual_tokenize(texts[-1], reduce_len=True)


['@rickrau', 'mind', 'BLOOOWWWN', 'by', 'latest', 'lex', ':*)', '!', '!', '!']

## Normalizing your vocabulary

Doctor e doctor tem significados diferentes

a Normalização se trata de colocar as palavras em minusculas

In [ ]:
# em python facilmente você pode normalizar facilmente usando list comprehension

tokens = ["House", "Visitor", "Center"]
normalized_tokens = [x.lower() for x in tokens] # lower() -> minusculas
print(normalized_tokens)

['house', 'visitor', 'center']


Infelizmente, essa abordagem também "normalizará" muitas letras maiúsculas significativas, além da capitalização menos significativa da primeira palavra da frase que você pretendia normalizar. Uma abordagem melhor para a normalização de maiúsculas e minúsculas é colocar em minúscula apenas a primeira palavra de uma frase e permitir que todas as outras palavras mantenham sua capitalização.

### Stemming

In [ ]:
# simples stemmer que pode lidar com S's finais.
def stem(phrase):
     return ' '.join([re.findall('^(.*ss|.*?)(s)?$',
         word)[0][0].strip("'") for word in phrase.lower().split()])
stem('houses')

stem("Doctor House's calls")


'doctor house call'

Esta função funciona bem para casos regulares, mas não é capaz de resolver casos mais complexos. Por exemplo, as regras falhariam com palavras como dishesou heroes. Para casos mais complexos como estes, o pacote NLTK fornece outros lematizadores.

In [ ]:
# Porter
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
' '.join([stemmer.stem(w).strip("'") for w in
   "dish washer's fairly washed dishes".split()])

'dish washer fairli wash dish'

O lematizador Snowball é mais agressivo que o lematizador Porter.  Observe que deriva de 'fairly' para 'fair', o que é mais preciso do que o lematizador de Porter.

In [ ]:
# Snowball

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')
' '.join([stemmer.stem(w).strip("'") for w in
   "dish washer's fairly washed dishes".split()])


'dish washer fair wash dish'

### Lemmatization
How can you identify word lemmas in Python?

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("better")

'better'

In [ ]:
lemmatizer.lemmatize("better", pos="a")  #


'good'

In [ ]:
lemmatizer.lemmatize("good", pos="a")


'good'

In [ ]:
stemmer.stem('goodness')


'good'

Você pode implementar facilmente a lematização no spaCy da seguinte maneira:

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("better good goods goodness best")
for token in doc:
  print(token.text, token.lemma_)

better well
good good
goods good
goodness goodness
best good


## Analise de Sentimento

Abordagem baseada em regras (VADER)

In [ ]:
pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.8 MB/s eta 0:00:00


SentimentIntensityAnalyzer.lexicon contém aquele dicionário de tokens e suas pontuações sobre os quais falamos.


É melhor que um tokenizer seja bom em lidar com pontuação e emoticons (emojis) para que o VADER funcione bem. Afinal, os emoticons são projetados para transmitir muito sentimento (emoção).


Se você usar um lematizador (ou lematizador) em seu pipeline, precisará aplicar esse lematizador também ao léxico VADER, combinando as pontuações de todas as palavras que vão juntas em um único radical ou lema.


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sa = SentimentIntensityAnalyzer()
sa.lexicon

{'$:': -1.5,
 '%)': -0.4,
 '%-)': -1.5,
 '&-:': -0.4,
 '&:': -0.7,
 "( '}{' )": 1.6,
 '(%': -0.9,
 "('-:": 2.2,
 "(':": 2.3,
 '((-:': 2.1,
 '(*': 1.1,
 '(-%': -0.7,
 '(-*': 1.3,
 '(-:': 1.6,
 '(-:0': 2.8,
 '(-:<': -0.4,
 '(-:o': 1.5,
 '(-:O': 1.5,
 '(-:{': -0.1,
 '(-:|>*': 1.9,
 '(-;': 1.3,
 '(-;|': 2.1,
 '(8': 2.6,
 '(:': 2.2,
 '(:0': 2.4,
 '(:<': -0.2,
 '(:o': 2.5,
 '(:O': 2.5,
 '(;': 1.1,
 '(;<': 0.3,
 '(=': 2.2,
 '(?:': 2.1,
 '(^:': 1.5,
 '(^;': 1.5,
 '(^;0': 2.0,
 '(^;o': 1.9,
 '(o:': 1.6,
 ")':": -2.0,
 ")-':": -2.1,
 ')-:': -2.1,
 ')-:<': -2.2,
 ')-:{': -2.1,
 '):': -1.8,
 '):<': -1.9,
 '):{': -2.3,
 ');<': -2.6,
 '*)': 0.6,
 '*-)': 0.3,
 '*-:': 2.1,
 '*-;': 2.4,
 '*:': 1.9,
 '*<|:-)': 1.6,
 '*\\0/*': 2.3,
 '*^:': 1.6,
 ',-:': 1.2,
 "---'-;-{@": 2.3,
 '--<--<@': 2.2,
 '.-:': -1.2,
 '..###-:': -1.7,
 '..###:': -1.9,
 '/-:': -1.3,
 '/:': -1.3,
 '/:<': -1.4,
 '/=': -0.9,
 '/^:': -1.0,
 '/o:': -1.4,
 '0-8': 0.1,
 '0-|': -1.2,
 '0:)': 1.9,
 '0:-)': 1.4,
 '0:-3': 1.5,
 '0:03': 1.9,
 '

In [ ]:
[(tok, score) for tok, score in sa.lexicon.items() if " " in tok]

[("( '}{' )", 1.6),
 ("can't stand", -2.0),
 ('fed up', -1.8),
 ('screwed up', -1.5)]

Dos 7.500 tokens definidos no VADER, apenas 3 contêm espaços, e apenas 2 deles são na verdade n-gramas; o outro é um emoticon para “beijo”.

In [ ]:
# O algoritmo VADER considera a intensidade da polaridade do sentimento
# em três pontuações separadas (positiva, negativa e neutra) e depois
# as combina em um sentimento composto de positividade.
sa.polarity_scores(text=\
   "Python is very readable and it's great for NLP.")


{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'compound': 0.6249}

In [ ]:
sa.polarity_scores(text=\
   "Python is not a bad choice for most applications.")

{'neg': 0.0, 'neu': 0.737, 'pos': 0.263, 'compound': 0.431}

Vamos ver o desempenho dessa abordagem baseada em regras para as declarações de exemplo que mencionamos anteriormente.

In [ ]:
corpus = ["Absolutely perfect! Love it! :-) :-) :-)",
           "Horrible! Completely useless. :(",
           "It was OK. Some good and some bad things."]

In [ ]:
for doc in corpus:
  scores = sa.polarity_scores(doc)
  print('{:+}: {}'.format(scores['compound'], doc))

+0.9428: Absolutely perfect! Love it! :-) :-) :-)
-0.8768: Horrible! Completely useless. :(
-0.1531: It was OK. Some good and some bad things.
